<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект. Анализ вакансий на HeadHunter

## Юнит 1. Работа с базой данных из Python

---

В реальной деятельности дата-сайентисту требуется не только писать запрос к данным, но и затем обрабатывать его результаты с помощью *Python*. Для решения таких задач требуется некоторое средство, которое будет связывать *Python* и *PostgreSQL* так, чтобы мы могли с помощью *Python* отправлять запросы в *Postgres* и принимать оттуда результаты.

Именно таким средством является пакет [psycopg2](https://www.psycopg.org/docs/), о котором мы сейчас и поговорим.

### Установка psycopg2

```linux
pip install psycopg2
```

Для подключения нам потребуются следующие данные:

* dbname — название базы, к которой нужно подключиться;
* user — имя пользователя в СУБД;
* password — пароль;
* host — адрес, по которому нужно подключиться;
* port — порт, к которому нужно подключиться (по умолчанию равен 5432).

Для нашей базы эти параметры сохранены в текущей директории в файле `config.ini`, который добавлен в список исключений `.gitignore` 

### Импорт библиотек. Соединение с БД

❗ При вызове функции `read_sql_query` выводилось предупреждение:
```
UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
```
Поэтому для соединения с БД SQL вместо `psycopg2` использована библиотека `sqlalchemy`

In [ ]:
from bs4 import BeautifulSoup
import configparser
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import requests
import warnings
from IPython.display import display, HTML

In [168]:
# Получение сведений об авторизации.
config = configparser.ConfigParser()
config.read("config.ini")

# connection = psycopg2.connect(
#     dbname=config.get("postgresql", 'DBNAME'),
#     user=config.get("postgresql", "USER"),
#     host=config.get("postgresql", "HOST"),
#     password=config.get("postgresql", "PASSWORD"),
#     port=config.get("postgresql", "PORT"),
# )

# Исключение предупреждающих сообщений из выходных данных ячеек (требуется при использования библиотеки psycopg)
# warnings.simplefilter(action='ignore', category=FutureWarning)
# warnings.simplefilter(action='ignore', category=UserWarning)

# ----------------------------------------------------------------------------
# Для соединения с БД вместо psycopg2 использована библиотека sqlalchemy,
# которая полностью совместима с pandas и не выводится никаких предупреждений
# ----------------------------------------------------------------------------

# Формирование строки подключения
db_params = {
    "drivername": "postgresql",
    "username": config.get("postgresql", "USER"),
    "password": config.get("postgresql", "PASSWORD"),
    "host": config.get("postgresql", "HOST"),
    "port": config.get("postgresql", "PORT"),
    "database": config.get("postgresql", "DBNAME"),
}

# Формируем строку подключения через SQLAlchemy
db_url = URL.create(**db_params)


# Создание SQLAlchemy engine
engine = create_engine(db_url)

Запрос:

In [169]:
n = 10
query = f"""
   SELECT *
     FROM vacancies
    LIMIT {n}
"""

Выполнение запроса

In [170]:
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)
connection.close()
df

,id,name,key_skills,schedule,experience,employment,salary_from,salary_to,area_id,employer_id
0,55312386,Компьютерный Мастер,Пользователь ПК\tРабота в команде\tРемонт ноут...,Полный день,Нет опыта,Полная занятость,64000.0,NaN,1516,5724811
1,55843014,Системный администратор,Средства криптографической защиты информации\t...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,130,4903713
2,54525964,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,160,69961
3,54525965,Lead Java Developer to Poland,Spring Framework\tSQL\tHibernate ORM\tJava\tGit,Удаленная работа,От 3 до 6 лет,Полная занятость,NaN,NaN,159,69961
4,55354053,Специалист службы поддержки с техническими зна...,None,Удаленная работа,Нет опыта,Частичная занятость,15000.0,NaN,1955,1740
5,55906072,Корректор,Корректура текстов\tГрамотность\tРабота в кома...,Полный день,Нет опыта,Полная занятость,NaN,NaN,2323,584934
6,55523207,React Front-end Middle Developer,JavaScript\tTypeScript\tCSS3\tNode.js\tHTML5,Гибкий график,От 1 года до 3 лет,Полная занятость,NaN,NaN,2759,204511
7,55050261,Инженер-программист,Java SE\tSpring Framework\tSpring Boot\tGit\tS...,Полный день,Нет опыта,Полная занятость,40000.0,60000.0,49,4448636
8,55583432,Системный администратор,Linux\tАдминистрирование сетевого оборудования...,Полный день,От 1 года до 3 лет,Полная занятость,50000.0,NaN,3,55126
9,55312414,Ведущий специалист группы разработки систем,1С программирование\t1С: Предприятие 8\t1С: До...,Полный день,От 3 до 6 лет,Полная занятость,NaN,NaN,47,1413754


 <a name="anchor2"></a>
## Юнит 2. Введение. Знакомство с данными
---

Представьте, что вы устроились на работу в кадровое агентство, которое подбирает вакансии для IT-специалистов. Ваш первый проект — создание модели машинного обучения, которая будет рекомендовать вакансии клиентам агентства, претендующим на позицию Data Scientist. Сначала вам необходимо понять, что из себя представляют данные и насколько они соответствуют целям проекта. В литературе эта часть работы над ML-проектом называется Data Understanding, или анализ данных.

Наш проект включает в себя несколько этапов:

* знакомство с данными;
* предварительный анализ данных;
* детальный анализ вакансий;
* анализ работодателей;
* предметный анализ.

Каждая из частей будет состоять из блока практических заданий, которые вам необходимо выполнить в своих Jupyter-ноутбуках, и контрольных вопросов на платформе, которые проверяются автоматически.

Также вам предстоит отправить свой код ментору для код-ревью. Вам будет предоставлен ноутбук-шаблон и требования, согласно которым вы должны оформить своё решение.

Требования к оформлению ноутбука-решения:

* Решение оформляется только в Jupyter Notebook.
* Решение оформляется в соответствии с [ноутбуком-шаблоном](https://lms.skillfactory.ru/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block@Project_2_%D0%9D%D0%BE%D1%83%D1%82%D0%B1%D1%83%D0%BA_%D1%88%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD.ipynb).
* Каждое задание выполняется в отдельной ячейке, выделенной под задание (в шаблоне они помечены как ваш код здесь). Не следует создавать много ячеек для решения задачи — это провоцирует неудобства при проверке.
* Текст SQL-запросов и код на Python должны быть читаемыми. Не забывайте про отступы в SQL-коде.
* Выводы по каждому этапу оформляются в формате Markdown в отдельной ячейке (в шаблоне они помечены как ваши выводы здесь).
* Выводы можно дополнительно проиллюстрировать с помощью графиков. Они оформляются в соответствии с теми правилами, которые мы приводили в модуле по визуализации данных.
* Не забудьте удалить ячейку с данными соединения перед фиксацией работы в GitHub.



Все необходимые таблицы находятся в схеме public базы данных project_sql (именно эту базу вам необходимо указать в параметре dbname при подключении).

То есть параметры подключения будут следующими: см. файл `config.ini`

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_1.png)

Познакомимся с каждой таблицей.

vacancies

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_2.png)


Зарплатная вилка — это верхняя и нижняя граница оплаты труда в рублях (зарплаты в других валютах уже переведены в рубли). Соискателям она показывает, в каком диапазоне компания готова платить сотруднику на этой должности.

areas

Таблица-справочник, которая хранит код региона и его название.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_3.png)

employers

Таблица-справочник со списком работодателей.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_4.png)

industries

Таблица-справочник вариантов сфер деятельности работодателей.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_5.png)

employers_industries

Дополнительная таблица, которая существует для организации связи между работодателями и сферами их деятельности.

Эта таблица нужна нам, поскольку у одного работодателя может быть несколько сфер деятельности (или работодатели могут вовсе не указать их). Для удобства анализа необходимо хранить запись по каждой сфере каждого работодателя в отдельной строке таблицы.

![](images/asset-v1%20SkillFactory+DST-3.0+28FEB2021+type@asset+block@SQL_pj2_2_6.png)

<a name="anchor3"></a>
## Юнит 3. Предварительный анализ данных
---

### Задание 3.1

> 1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [152]:
# текст запроса
query = f'''
   SELECT count(*)
     FROM vacancies
'''

# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [153]:
# результат запроса
text = f"<p style='font-size:25px;'>Ответ (3.1)<br><br>В базе содержится {df.iloc[0, 0]} вакансий.</p>"
display(HTML(text))

### Задание 3.2

> 2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [101]:
# текст запроса
query = f'''
   SELECT count(*)
     FROM employers
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [102]:
# результат запроса
text = f"<p style='font-size:25px;'>Ответ (3.2)<br><br>В базе содержится {df.iloc[0, 0]} работодателей.</p>"
display(HTML(text))

### Задание 3.3

> 3. Посчитате с помощью запроса количество регионов (таблица areas).

In [99]:
# текст запроса
query = f'''
   SELECT count(*)
     FROM areas
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [100]:
# результат запроса
text = f"<p style='font-size:25px;'>Ответ (3.3)<br><br>В базе содержится {df.iloc[0, 0]} региона.</p>"
display(HTML(text))

### Задание 3.4

> 4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [97]:
# текст запроса
query = f'''
   SELECT count(*)
     FROM industries
'''
# df = pd.read_sql_query(query, connection)7
df = pd.read_sql_query(query, engine)

In [98]:
# результат запроса
text = f"<p style='font-size:25px;'>Ответ (3.4)<br><br>В базе содержится {df.iloc[0, 0]} сферы деятельности.</p>"
display(HTML(text))

<a name="anchor4"></a>
## Юнит 4. Детальный анализ ваканский
---

### Задание 4.1

> 1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [94]:
# текст запроса
query = f'''
   SELECT a.name AS area,
          count(v.id) AS cnt
     FROM areas AS a
     JOIN vacancies AS v ON a.id = v.area_id
 GROUP BY a.id
 ORDER BY 2 DESC
    LIMIT 5
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [95]:
text = f"<p style='font-size:25px;'>Ответ (4.1)<br><br>Количество вакансий по регионам (5-ка лидеров)</p>"
display(HTML(text))
display(df)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


### Задание 4.2

> 2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [91]:
# текст запроса
query = f'''
   SELECT count(*)
     FROM vacancies AS v
    WHERE v.salary_from IS NOT NULL
       OR v.salary_to IS NOT NULL
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [ ]:
text = (
    f"<p style='font-size:25px;'>Ответ (4.2)<br><br>"
    f"Количество вакансий, где заполнено хотя бы одно из полей с зарплатой, составляет {df.iloc[0, 0]}.</p>"
)
display(HTML(text))

### Задание 4.3

> 3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [21]:
# текст запроса
query = f'''
   SELECT round(avg(salary_from))
     FROM vacancies
'''
# mean_salary_from = int(pd.read_sql_query(query, connection).iloc[0, 0])
mean_salary_from = int(pd.read_sql_query(query, engine).iloc[0, 0])

query = f'''
   SELECT round(avg(salary_to))
     FROM vacancies
'''
# mean_salary_to = int(pd.read_sql_query(query, connection).iloc[0, 0])
mean_salary_to = int(pd.read_sql_query(query, engine).iloc[0, 0])

In [81]:
text = f"<p style='font-size:25px;'>Ответ (4.3)</p>"
display(HTML(text))
print("Среднее значение зарплаты:")
print(f"    \u2022 для нижней границы составляет {mean_salary_from:,} \u20BD;".replace(",", " "))
print(f"    \u2022 для верхней границы составляет {mean_salary_to:,} \u20BD.".replace(",", " "))

Среднее значение зарплаты:
    • для нижней границы составляет 71 065 ₽;
    • для верхней границы составляет 110 537 ₽.


### Задание 4.4

> 4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [172]:
query = f'''
   SELECT schedule,
          employment,
          count(*)
     FROM vacancies
 GROUP BY schedule,
          employment
 ORDER BY 3 DESC
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

OperationalError: (psycopg2.OperationalError) server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

[SQL: 
   SELECT schedule,
          employment,
          count(*)
     FROM vacancies
 GROUP BY schedule,
          employment
 ORDER BY 3 DESC
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [24]:
df

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


In [171]:
text = f"""
    <p style="font-size:25px;">
        Ответ (4.4)<br><br>
        На втором месте по популярности находится график "{df.iloc[1, 0]} — {df.iloc[1, 1]}"
    </p>
"""
display(HTML(text))

> 5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [32]:
query = f'''
   SELECT experience,
          count(*)
     FROM vacancies
 GROUP BY experience
 ORDER BY 2
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [33]:
df

,experience,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


<a name="anchor5"></a>
## Юнит 5. Анализ работодателей
---

### Задание 5.1

> 1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [108]:
query= f'''
   SELECT e.name,
          count(*)
     FROM employers AS e
     JOIN vacancies AS v ON e.id = v.employer_id
 GROUP BY e.id,
          e.name
 ORDER BY 2 DESC
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [109]:
df.head(10)

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
5,ИК СИБИНТЕК,327
6,МТС,292
7,DataArt,247
8,Совкомбанк Технологии,204
9,Первый Бит,176


In [112]:
text = (
    f"<p style='font-size:25px;'>Ответ (5.1)<br><br>"
    f'На 1-м и 5-м месте по количеству вакансий находятся "{df['name'].iloc[0]}" и "{df['name'].iloc[4]}".'
)
display(HTML(text))
# print(f"")

### Задание 5.2

> 2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [113]:
query = f'''
    WITH areas_vacancies_cnt AS (
             SELECT a.id,
                    a.name AS name,
                    count(v.area_id) AS vacancies_cnt
               FROM areas AS a
          LEFT JOIN vacancies AS v ON a.id = v.area_id
           GROUP BY a.id,
                    a.name
          )
   SELECT av.name,
          av.vacancies_cnt,
          count(e.id) AS employers_cnt
     FROM areas_vacancies_cnt AS av
LEFT JOIN employers AS e ON e.area = av.id
 GROUP BY av.id,
          av.name,
          av.vacancies_cnt
   HAVING vacancies_cnt = 0
 ORDER BY 3 DESC
    -- LIMIT 1     
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [116]:
df

,name,vacancies_cnt,employers_cnt
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18
...,...,...,...
588,Козьмодемьянск,0,0
589,Струнино,0,0
590,Верхний Тагил,0,0
591,Кант,0,0


In [114]:
text = (
    f"<p style='font-size:25px;'>Ответ (5.2)<br><br>"
    f"Среди регионов, не имеющих вакансий, наибольшее количество работодателей в регионе \"{df.iloc[0, 0]}\""
)
display(HTML(text))
# print()

### Задание 5.3

> 3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [119]:
query = f'''
    select
        e.name as employer_name,
        count(distinct v.area_id) as areas_cnt
    from
        employers as e
        join vacancies as v on e.id = v.employer_id
    group by
        e.id,
        e.name
    order by
        2 DESC
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [120]:
text = (
    f"<p style='font-size:25px;'>Ответ (5.3)<br><br>"
    f"Максимальное количество регионов присутствия - {df.iloc[0, 1]} - у компании \"{df.iloc[0,0]}\""
)
display(HTML(text))

### Задание 5.4

> 4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [121]:
query = f'''
   SELECT count(DISTINCT e.id)
     FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
    WHERE ei.employer_id IS NULL
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [123]:
text = (
    f"<p style='font-size:25px;'>Ответ (5.4)<br><br>"
    f"Количество работодателей, у которых не указана сфера деятельности, составляет {df.iloc[0, 0]}."
)
display(HTML(text))

### Задание 5.5

> 5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [125]:
query = f'''
   SELECT e.name,
          count(DISTINCT ei.industry_id)
     FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
 GROUP BY e.id
   HAVING count(DISTINCT ei.industry_id) = 4
 ORDER BY 1
    LIMIT 3  
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [127]:
text = (
    f"<p style='font-size:25px;'>Ответ (5.5)<br><br>"
    "Компания, находящаяся на третьем месте в алфавитном списке (по названию) компаний,<br>"
    f'у которых указано четыре сферы деятельности, является "{df.iloc[2, 0]}"'
)
display(HTML(text))

### Задание 5.6

> 6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [128]:
source_industry = "Разработка программного обеспечения"
query = f'''
   SELECT count(DISTINCT e.id) AS количество_работодателей
     FROM employers e
     JOIN employers_industries ei ON e.id = ei.employer_id
     JOIN industries i ON ei.industry_id = i.id
    WHERE i.name = '{source_industry}'
;
'''
# df = pd.read_sql_query(query, connection)
df = pd.read_sql_query(query, engine)

In [131]:
text = (
    f"<p style='font-size:25px;'>Ответ (5.6)<br><br>"
    f"Количество работодателей, занимающихся разработкой программного обеспечения, составляет {df.iloc[0,0]}."
)
display(HTML(text))

### Задание 5.7

> 7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.
>
> Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 
> 
> Если возникнут трудности с этим заданием посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [155]:
# Получаем список городов-миллионников
url = "https://ru.wikipedia.org/wiki/Города-миллионеры_России"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
# Ищем всё с тегом table на листе.
table = soup.find_all("table")
# Опытным путем выяснилось, что искомая таблица имеет индекс 0.
rows = table[0].find_all("tr")[1:]
cities = [row.find_all("td")[1].text.strip() for row in rows]

Вариант решения № 1

In [ ]:
# Парметры для передачи в SQL запрос (кортежи с городами-миллионниками)
parameters = {
    "city_list_1": tuple(cities),
    "city_list_2": tuple(cities),
}

query = f"""
   SELECT name,
          cnt
          -- Подзапросы используются для сортировки (чтобы строка итогов была внизу),
          -- а остальная таблица была отсортирована по возрастанию количества вакансий
          -- как в примере в условии задания на платформе.
     FROM (
             SELECT a.name,
                    count(v.id) AS cnt,
                    1 AS order_col -- технический столбец для сортировки
               FROM areas AS a
               JOIN vacancies AS v ON a.id = v.area_id
               JOIN employers AS e ON e.id = v.employer_id
              WHERE a.name IN %(city_list_1)s
                AND e.name = 'Яндекс'
           GROUP BY a.id,
                    a.name
          UNION ALL -- формирование строки итогов
             SELECT 'Total' AS name,
                    count(v.id) AS cnt,
                    2 AS order_col -- технический столбец для сортировки
               FROM areas AS a
               JOIN vacancies AS v ON a.id = v.area_id
               JOIN employers AS e ON e.id = v.employer_id
              WHERE a.name IN %(city_list_2)s
                AND e.name = 'Яндекс'
          ) sub -- сортировка таблицы целиком
 ORDER BY order_col,
          cnt
;
"""

df = pd.read_sql_query(
    query,
    engine,
    params=parameters,
)

Вариант решения № 2 (более короткий)

In [ ]:
# Парметр для передачи в SQL запрос (кортеж с городами-миллионниками)
parameters = {"city_list": tuple(cities)}

query = f"""
WITH filtered_data AS (
             SELECT a.name AS city_name,
                    COUNT(v.id)::INTEGER AS cnt
               FROM areas AS a
               JOIN vacancies AS v ON a.id = v.area_id
               JOIN employers AS e ON e.id = v.employer_id
              WHERE a.name IN %(city_list)s
                AND e.name = 'Яндекс'
           GROUP BY a.name
           ORDER BY 2
          )
   SELECT city_name AS name,
          cnt
     FROM filtered_data
UNION ALL
   SELECT 'Total' AS name,
          SUM(cnt)::INTEGER AS cnt
     FROM filtered_data
;
"""

df = pd.read_sql_query(
    query,
    engine,
    params=parameters,
)

In [159]:
text = (
    f"<p style='font-size:25px;'>Ответ (5.7)<br><br>"
    "Количество вакансий компании \"Яндекс\" в городах-миллионниках"
)
display(HTML(text))
display(df)

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


<a name="anchor6"></a>
## Юнит 6. Предметный анализ

---

> 1. Сколько вакансий имеет отношение к данным?
>
> Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.
>
> *Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [67]:
query_6_1 = f'''
   SELECT count(DISTINCT v.id) AS cnt
     FROM vacancies AS v
    WHERE lower(v.name) LIKE '%data%'
       OR lower(v.name) LIKE '%данн%'
'''
df = pd.read_sql_query(query_6_1, connection)

In [68]:
print(f"В базе содержится {df.iloc[0, 0]} вакансия, имеющая отношение к данным.")

В базе содержится 1771 вакансия, имеющая отношение к данным.


> 2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
> Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
> * 'data scientist'
> * 'data science'
> * 'исследователь данных'
> * 'ML' (здесь не нужно брать вакансии по HTML)
> * 'machine learning'
> * 'машинн%обучен%'
> 
> **В следующих заданиях мы продолжим работать с вакансиями по этому условию.*
> 
> Считаем вакансиями для специалистов уровня Junior следующие:
> * в названии есть слово 'junior' *или*
> * требуемый опыт — Нет опыта *или*
> * тип трудоустройства — Стажировка.
 

In [5]:
query_6_2 = f'''
   SELECT count(DISTINCT v.id) AS cnt
     FROM vacancies AS v
    WHERE (
          lower(v.name) LIKE '%data scientist%'
       OR lower(v.name) LIKE '%data science%'
       OR lower(v.name) LIKE '%исследователь данных%'
       OR lower(v.name) LIKE '%machine learning%'
       OR lower(v.name) LIKE '%машинн%обучен%'
       OR (
          v.name LIKE '%ML%'
      AND v.name NOT LIKE '%HTML%'
          )
          )
      AND (
          lower(v.name) LIKE '%junior%'
       OR lower(v.experience) LIKE '%нет опыта%'
       OR lower(v.employment) LIKE '%стажировка%'
          )
'''
df = pd.read_sql_query(query_6_2, connection)

In [6]:
print(f"В базе содержится {df.iloc[0, 0]} вакансия для начинающего дата-сайентиста.")

В базе содержится 51 вакансия для начинающего дата-сайентиста.


> 3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?
> 
> **Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [7]:
query_6_3 = f'''
   SELECT count(DISTINCT v.id) AS cnt
     FROM vacancies AS v
    WHERE (
          lower(v.name) LIKE '%data scientist%'
       OR lower(v.name) LIKE '%data science%'
       OR lower(v.name) LIKE '%исследователь данных%'
       OR lower(v.name) LIKE '%machine learning%'
       OR lower(v.name) LIKE '%машинн%обучен%'
       OR (
          v.name LIKE '%ML%'
      AND v.name NOT LIKE '%HTML%'
          )
          )
      AND (
          lower(v.key_skills) LIKE '%sql%'
       OR lower(v.key_skills) LIKE '%postgres%'
          )
'''
df = pd.read_sql_query(query_6_3, connection)

In [8]:
print(f"В базе содержится {df.iloc[0, 0]} вакансия для дата-сайентиста, где в качестве ключевого навыка указан SQL или postgres.")

В базе содержится 201 вакансия для дата-сайентиста, где в качестве ключевого навыка указан SQL или postgres.


> 4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.
> 
> **Это можно сделать помощью запроса, аналогичного предыдущему.*

In [9]:
query_6_4 = f'''
   SELECT count(DISTINCT v.id) AS cnt
     FROM vacancies AS v
    WHERE (
          lower(v.name) LIKE '%data scientist%'
       OR lower(v.name) LIKE '%data science%'
       OR lower(v.name) LIKE '%исследователь данных%'
       OR lower(v.name) LIKE '%machine learning%'
       OR lower(v.name) LIKE '%машинн%обучен%'
       OR (
          v.name LIKE '%ML%'
      AND v.name NOT LIKE '%HTML%'
          )
          )
      AND lower(v.key_skills) LIKE '%python%'
      
'''
df = pd.read_sql_query(query_6_4, connection)

In [10]:
print(f"В базе содержится {df.iloc[0, 0]} вакансия для дата-сайентиста, где в качестве ключевого навыка указан Python.")

В базе содержится 351 вакансия для дата-сайентиста, где в качестве ключевого навыка указан Python.


> 5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
> Ответ округлите до двух знаков после точки-разделителя.

In [11]:
query_6_5 = f'''
   SELECT round(avg(cardinality), 2) AS avg_cnt
     FROM (
             SELECT cardinality(string_to_array(v.key_skills, '\t')) AS cardinality
               FROM vacancies AS v
              WHERE (
                    lower(v.name) LIKE '%data scientist%'
                 OR lower(v.name) LIKE '%data science%'
                 OR lower(v.name) LIKE '%исследователь данных%'
                 OR lower(v.name) LIKE '%machine learning%'
                 OR lower(v.name) LIKE '%машинн%обучен%'
                 OR (
                    v.name LIKE '%ML%'
                AND v.name NOT LIKE '%HTML%'
                    )
                    )
          ) AS subquery
;
'''
df = pd.read_sql_query(query_6_5, connection)

In [12]:
print(f"В вакансиях для DS в среднем указывают {df.iloc[0, 0]} ключевых навыка.")

В вакансиях для DS в среднем указывают 6.41 ключевых навыка.


> 6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 
> 
> При решении задачи примите во внимание следующее:
> 1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
> 2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
> 3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`
> 
> Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [15]:
query_6_6 = f'''
   SELECT v.experience AS experience,
          avg(
          (
          coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, salary_from)
          ) / 2
          ) AS avg_salary
     FROM vacancies AS v
    WHERE (
          lower(v.name) LIKE '%data scientist%'
       OR lower(v.name) LIKE '%data science%'
       OR lower(v.name) LIKE '%исследователь данных%'
       OR lower(v.name) LIKE '%machine learning%'
       OR lower(v.name) LIKE '%машинн%обучен%'
       OR (
          v.name LIKE '%ML%'
      AND v.name NOT LIKE '%HTML%'
          )
          )
      AND (
          v.salary_from IS NOT NULL
       OR v.salary_to IS NOT NULL
          )
 GROUP BY v.experience
;
'''
df = pd.read_sql_query(query_6_6, connection)

mask = df['experience'] == 'От 3 до 6 лет'
source_salary = round(df[mask].iloc[0, 1])

In [16]:
df

,experience,avg_salary
0,Нет опыта,74642.857143
1,От 1 года до 3 лет,139674.750000
2,От 3 до 6 лет,243114.666667


In [17]:
print(
    "Дата-сайентист с опытом работы от 3 до 6 лет может рассчитывать на среднюю зарплату "
    f"{source_salary:,} \u20BD.".replace(
        ",", " "
    )
)

Дата-сайентист с опытом работы от 3 до 6 лет может рассчитывать на среднюю зарплату 243 115 ₽.


In [18]:
connection.close()

<a name="anchor7"></a>
## Юнит 7. Финальное задание и итоги
---